### Dependencies

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from numpy.random import randn
from scipy.stats import pearsonr
from numpy import corrcoef

from sklearn.metrics import r2_score

from sklearn.datasets import make_regression
from matplotlib import pyplot

### Create a dataset to test Linear regression and ridge regression

##### Add some noise to dataset that can be resolved with regularization such as multicollinearity, lesser number of data points.

In [18]:
def generate_dataset(n_samples, n_features, effective_rank, noise, random_state=6, coef=True):
    # generate regression dataset
    X, y, coef = make_regression(n_samples=n_samples, 
                                 n_features=n_features,  
                                 effective_rank=effective_rank,
                                 noise=noise,
                                 random_state=random_state,
                                 coef=coef)   
    # add later:
    # bias, n_informative
    X = pd.DataFrame(X)
    y = pd.Series(y)
    X.columns = [f'col_{i+1}' for i in range(len(X.columns))]
    
    return X, y, coef


# noise --- The standard deviation of the gaussian noise applied to the output.
# effective_rank --- The approximate number of singular vectors required to explain most of the input data by linear combinations. 
# Using this kind of singular spectrum in the input allows the generator to reproduce the correlations often observed in practice.

In [19]:
X, y, coef_data = generate_dataset(n_samples=5000, n_features=10, effective_rank=1, noise=0.3)

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
X_norm = pd.DataFrame(X_norm)
X_norm.columns = [f'col_{i+1}' for i in range(len(X.columns))]
# X.head()

### Train on Scikit-learn algorithms

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.98, random_state=6)

In [21]:
linear = LinearRegression()

print("----- Linear Regression -----")
val_scores = cross_val_score(linear, X_train, y_train, cv=5)
print(f'Cross validation scores : {val_scores}')

linear.fit(X_train, y_train)
test_score = linear.score(X_test, y_test)
print(f'Test score : {test_score}')

----- Linear Regression -----
Cross validation scores : [0.9625622  0.95329889 0.90517589 0.9482027  0.92770664]
Test score : 0.9521110550872984


In [22]:
ridge = Ridge(alpha=1)

print("----- Ridge Regression -----")
val_scores = cross_val_score(ridge, X_train, y_train, cv=5)
print(f'Cross validation scores : {val_scores}')

ridge.fit(X_train, y_train)
test_score = ridge.score(X_test, y_test)
print(f'Test score : {test_score}')

----- Ridge Regression -----
Cross validation scores : [0.96567773 0.94949791 0.90408329 0.95208436 0.92852214]
Test score : 0.953313396380002


### Training on custom algorithms

In [37]:
# generate dataset
X, y, coef_data = generate_dataset(n_samples=5000, n_features=10, effective_rank=1, noise=0.3)

# normalize features
X_norm = pd.DataFrame()
for col in X.columns:
    X_norm[col] = normalize(X[col])
    
# split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.98, random_state=6)

In [34]:
def normalize(col):
    '''
    normalize each variable between 0 to 1
    with mean zero and std. deviation equal to 1 
    '''
    mean = col.mean()
    std_dev = col.std()
    col = [(x - mean)/std_dev for x in col]
    
    return col

def score(y_true, y_pred):
    score = r2_score(y_true, y_pred)
    return score

def predict(feat, coef):
    '''
    Make prediction for a row (instance)
    '''
    y_pred = coef[0]
    
    for i in range(feat.shape[0]):
        y_pred += coef[i+1] * feat[i]
    return y_pred


def sgd_linear(train_X, label, lr, n_epoch):
  
    coef = [0.0 for i in range(train_X.shape[1] + 1)]
    
    for epoch in range(n_epoch):
        total_loss = 0
        
        for lbl, (idx, x) in zip(label, train_X.iterrows()):
            y_pred = predict(x, coef)
            loss = y_pred - lbl
            total_loss += loss**2
            coef[0] = coef[0] - lr * loss    # update bias coefficient
            
            for i in range(train_X.shape[1]):
                coef[i+1] = coef[i+1] - lr * loss * x[i]    # update features' coefficients
                
#         print('>epoch=%d, learning_rate=%.3f, total_loss=%.3f' % (epoch, lr, total_loss))
    return coef



def sgd_ridge(train_X, label, lr, n_epoch, alpha):
  
    coef = [0.0 for i in range(train_X.shape[1] + 1)]
    
    for epoch in range(n_epoch):
        total_loss = 0
        
        for lbl, (idx, x) in zip(label, train_X.iterrows()):
            y_pred = predict(x, coef)
            loss = y_pred - lbl
            total_loss += loss**2
            coef[0] = coef[0] - lr * loss    # update bias coefficient
            
            for i in range(train_X.shape[1]):
                coef[i+1] = coef[i+1] - lr * ((loss * x[i]) + (alpha * coef[i+1]))    # update features' coefficients
             
    return coef

### Batch Gradient Descent 

In [76]:
def get_derivative_bias(label, train_X, coef):
    
    derivative = 0
    for lbl, (idx, x) in zip(label, train_X.iterrows()):
        y_pred = predict(x, coef)
        loss = y_pred - lbl
        der = loss
        derivative += der
        
    return derivative/train_X.shape[0]


def get_derivative_coef(label, train_X, coef, j):
    
    derivative = 0
    for lbl, (idx, x) in zip(label, train_X.iterrows()):
        y_pred = predict(x, coef)
        loss = y_pred - lbl
        der = loss * x[j]
        derivative += der
        
    return derivative/train_X.shape[0]

def get_derivative_coef_bgd(label, train_X, coef, alpha, j):
    
    derivative = 0
    for lbl, (idx, x) in zip(label, train_X.iterrows()):
        y_pred = predict(x, coef)
        loss = y_pred - lbl
        der = loss * x[j]
        derivative += der
        
    derivative += alpha * coef[j+1]
    return derivative/train_X.shape[0]


def bgd_linear(train_X, label, lr, n_epoch):
  
    coef = [0.0 for i in range(train_X.shape[1] + 1)]
    
    for epoch in range(n_epoch):
        
        # get bias derivative, then update bias
        derivative_bias = get_derivative_bias(label, train_X, coef)
        coef[0] = coef[0] - lr * derivative_bias
        
        for j in range(train_X.shape[1]):
            
            # get derivative w.r.t j-th coef, then update coef
            derivative_coef = get_derivative_coef(label, train_X, coef, j)
            coef[j+1] = coef[j+1] - lr * derivative_coef
                        
    return coef


def bgd_ridge(train_X, label, lr, n_epoch, alpha):
  
    coef = [0.0 for i in range(train_X.shape[1] + 1)]
    
    for epoch in range(n_epoch):
        
        # get bias derivative, then update bias
        derivative_bias = get_derivative_bias(label, train_X, coef)
        coef[0] = coef[0] - lr * derivative_bias
        
        for j in range(train_X.shape[1]):
            
            # get derivative w.r.t j-th coef, then update coef
            derivative_coef = get_derivative_coef_bgd(label, train_X, coef, alpha, j)
            coef[j+1] = coef[j+1] - lr * derivative_coef
                        
    return coef

In [77]:
# train data
# coef_sgd_linear = sgd_linear(X_train, y_train, 0.01, 10)
# coef_sgd_ridge = sgd_ridge(X_train, y_train, 0.01, 10, 1)

In [87]:
# coef_bgd_linear = bgd_linear(X_train, y_train, 0.01, 400)    # try increasing learning rate
coef_bgd_ridge = bgd_ridge(X_train, y_train, 0.02, 400, 1)

### Evaluate algorithms

In [83]:
# Evaluate linear reg - SGD
# y_preds_linear_sgd = []
# for _, feat in X_test.iterrows():
#     y_pred = predict(feat, coef_sgd_linear)
#     y_preds_linear_sgd.append(y_pred)
    
# sgd_score = score(y_test, y_preds_linear_sgd)
# print(f'SGD Score : {sgd_score}')


# Evaluate linear reg - BGD
y_preds_linear_bgd = []
for _, feat in X_test.iterrows():
    y_pred = predict(feat, coef_bgd_linear)
    y_preds_linear_bgd.append(y_pred)
    
bgd_score = score(y_test, y_preds_linear_bgd)
print(f'BGD Score : {bgd_score}')

BGD Score : 0.9432390741231034


In [88]:
# # Train ridge regression
# coef_ridge = coef_sgd_ridge(X_train, y_train, 0.001, 400, 1)

# # Evaluate ridge reg
# y_preds_ridge = []
# for _, feat in X_test.iterrows():
#     y_pred = predict(feat, coef_ridge)
#     y_preds_ridge.append(y_pred)
    
# score(y_test, y_preds_ridge)


# Evaluate ridge reg
y_preds_ridge_bgd = []
for _, feat in X_test.iterrows():
    y_pred = predict(feat, coef_bgd_ridge)
    y_preds_ridge_bgd.append(y_pred)
    
score(y_test, y_preds_ridge_bgd)

0.9538302014993534

### Learning points

1. Wrote stochastic gradient descent algorithm from scratch
2. Exploding loss values (There was no error when tested with scikit-learn libraries)
3. Why batch GD might be more effective than SGD?
  (SGD fluctuates too much before converging, not enough data)
4. You can mention performance difference SGD vs BGD

In [ ]:
# pyplot.scatter(X[:, 2], y)
# pyplot.show()

# from scipy.linalg import eigh, cholesky
# from scipy.stats import norm

# num_samples = 100
# r = np.array([
#         [  3.40, -2.75, -2.00],
#         [ -2.75,  5.50,  1.50],
#         [ -2.00,  1.50,  1.25]
#     ])

# c = cholesky(r, lower=True)
# x = norm.rvs(size=(3, num_samples))
# y = np.dot(c, x)
# y